Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.867418
Minibatch accuracy: 12.5%
Validation accuracy: 13.2%
Minibatch loss at step 50: 1.889260
Minibatch accuracy: 31.2%
Validation accuracy: 50.5%
Minibatch loss at step 100: 1.243804
Minibatch accuracy: 50.0%
Validation accuracy: 65.5%
Minibatch loss at step 150: 0.733014
Minibatch accuracy: 75.0%
Validation accuracy: 75.0%
Minibatch loss at step 200: 0.831648
Minibatch accuracy: 68.8%
Validation accuracy: 76.3%
Minibatch loss at step 250: 0.962198
Minibatch accuracy: 62.5%
Validation accuracy: 76.8%
Minibatch loss at step 300: 0.618523
Minibatch accuracy: 68.8%
Validation accuracy: 78.2%
Minibatch loss at step 350: 0.135086
Minibatch accuracy: 93.8%
Validation accuracy: 78.1%
Minibatch loss at step 400: 0.261251
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Minibatch loss at step 450: 0.397870
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Minibatch loss at step 500: 0.570161
Minibatch accuracy: 93.8%
Validation accuracy: 80.7%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [10]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.501850
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.248790
Minibatch accuracy: 6.2%
Validation accuracy: 22.0%
Minibatch loss at step 100: 1.307972
Minibatch accuracy: 56.2%
Validation accuracy: 39.5%
Minibatch loss at step 150: 0.997140
Minibatch accuracy: 56.2%
Validation accuracy: 68.7%
Minibatch loss at step 200: 0.740351
Minibatch accuracy: 68.8%
Validation accuracy: 74.1%
Minibatch loss at step 250: 1.119201
Minibatch accuracy: 62.5%
Validation accuracy: 77.4%
Minibatch loss at step 300: 0.466639
Minibatch accuracy: 93.8%
Validation accuracy: 79.3%
Minibatch loss at step 350: 0.280301
Minibatch accuracy: 87.5%
Validation accuracy: 76.1%
Minibatch loss at step 400: 0.249330
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.312289
Minibatch accuracy: 93.8%
Validation accuracy: 81.1%
Minibatch loss at step 500: 0.570251
Minibatch accuracy: 87.5%
Validation accuracy: 81.4%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [43]:
patch_size = 5
depth0 = 20
depth1 = 50
depth2 = 50
#for depth2 = 20
#hidden3_size = 980
#for depth2 = 50
hidden3_size = 2450
#for depth2 = 25
#hidden3_size = 19600
hidden4_size = 64

graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    #print("Shape of 1 training data batch placeholder: {}".format(np.shape(tf_train_dataset)))
    #print("Shape of 1 training label batch placeholder: {}".format(np.shape(tf_train_labels)))

    w0 = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth0], stddev=0.1))
    b0 = tf.Variable(tf.zeros([depth0]))
    
    w1 = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth0, depth1], stddev=0.1))
    b1 = tf.Variable(tf.zeros([depth1]))
    #print("\nShape of weights 1: {}".format(np.shape(w1)))
    #print("Shape of biases 1: {}".format(np.shape(b1)))

    w2 = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth1, depth2], stddev=0.1))
    b2 = tf.Variable(tf.zeros([depth2]))
    #print("\nShape of weights 2: {}".format(np.shape(w2)))
    #print("Shape of biases 2: {}".format(np.shape(b2)))
    
    w3 = tf.Variable(tf.truncated_normal(
      [hidden3_size, hidden4_size], stddev=0.1))
    b3 = tf.Variable(tf.zeros([hidden4_size]))
    #print("\nShape of weights 3: {}".format(np.shape(w3)))
    #print("Shape of biases 3: {}".format(np.shape(b3)))
    
    w4 = tf.Variable(tf.truncated_normal(
      [hidden4_size, num_labels], stddev=0.1))
    b4 = tf.Variable(tf.zeros([num_labels]))
    #print("\nShape of weights 4: {}".format(np.shape(w4)))
    #print("Shape of biases 4: {}".format(np.shape(b4)))
    
    # Model.
    def model(data):
        conv0 = tf.nn.conv2d(data, w0, [1, 1, 1, 1], padding='SAME')
        hidden0 = tf.nn.relu(conv0 + b0)
        
        conv1 = tf.nn.conv2d(hidden0, w1, [1, 1, 1, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + b1)
        pool1 = tf.nn.max_pool(hidden1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        #print("\nShape of conv 1: {}".format(np.shape(conv1)))
        #print("Shape of hidden 1: {}".format(np.shape(hidden1)))
        #print("Shape of pool 1: {}".format(np.shape(pool1)))

        conv2 = tf.nn.conv2d(pool1, w2, [1, 1, 1, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + b2)
        pool2 = tf.nn.max_pool(hidden2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        #print("\nShape of conv 2: {}".format(np.shape(conv2)))
        #print("Shape of hidden 2: {}".format(np.shape(hidden2)))
        #print("Shape of pool 2: {}".format(np.shape(pool2)))

        shape = pool2.get_shape().as_list()

        flatten1 = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        #print("Shape of fully connected layer 1: {}".format(np.shape(flatten1)))
        flatten2 = tf.nn.relu(tf.matmul(flatten1, w3) + b3)
        #print("Shape of fully connected layer 2: {}".format(np.shape(flatten2)))
        output = tf.matmul(flatten2, w4) + b4
        #print("Shape of output: {}".format(np.shape(output)))
        return output

    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [ ]:
num_steps = 40001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 2000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
              valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.080957
Minibatch accuracy: 6.2%
Validation accuracy: 10.4%
Minibatch loss at step 2000: 0.309809
Minibatch accuracy: 81.2%
Validation accuracy: 86.2%
Minibatch loss at step 4000: 0.070930
Minibatch accuracy: 100.0%
Validation accuracy: 88.2%
Minibatch loss at step 6000: 0.079638
Minibatch accuracy: 100.0%
Validation accuracy: 88.2%
Minibatch loss at step 8000: 0.622517
Minibatch accuracy: 81.2%
Validation accuracy: 89.4%
Minibatch loss at step 10000: 0.921880
Minibatch accuracy: 68.8%
Validation accuracy: 89.5%
Minibatch loss at step 12000: 0.650085
Minibatch accuracy: 81.2%
Validation accuracy: 89.7%
Minibatch loss at step 14000: 0.062509
Minibatch accuracy: 100.0%
Validation accuracy: 90.1%
Minibatch loss at step 16000: 0.164966
Minibatch accuracy: 93.8%
Validation accuracy: 90.3%
Minibatch loss at step 18000: 0.259045
Minibatch accuracy: 87.5%
Validation accuracy: 90.5%
Minibatch loss at step 20000: 0.203398
Minibatch accuracy: 93.8%
Validatio